# EV Charging Load Forecasting with LSTM

## 1. Objective
**Goal:** Predict future Hourly Charging Load based on historical load, weather, and traffic data.
**Model:** Multivariate LSTM (Long Short-Term Memory) Neural Network.
**Metrics:** MAE (Mean Absolute Error), RMSE (Root Mean Squared Error).

## 2. Setup & Data Preparation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import os

# Configuration
SEQ_LENGTH = 24  # Look back 24 hours
BATCH_SIZE = 64
EPOCHS = 50
LEARNING_RATE = 0.001
HIDDEN_SIZE = 64
NUM_LAYERS = 2

# Paths
DATA_DIR = '../data'
FILE_HOURLY = os.path.join(DATA_DIR, 'Dataset_2_Hourly_EV_per_user.csv')
FILE_TRAFFIC = os.path.join(DATA_DIR, 'Dataset 6_Local traffic distribution.csv')
FILE_WEATHER = os.path.join(DATA_DIR, 'Norway_Trondheim_ExactLoc_Weather.csv')

## 3. Data Loading and Merging
We prepare a single dataframe containing the Target (`total_load`) and all Features (Traffic, Weather, Time).

In [ ]:
def prepare_dataset():
    # 1. Load Hourly Load
    df_hourly = pd.read_csv(FILE_HOURLY, sep=';', decimal=',')
    df_hourly['datetime'] = pd.to_datetime(df_hourly['date_from'], format='%d.%m.%Y %H:%M')
    load_cols = [c for c in df_hourly.columns if 'kW' in c]
    df_hourly[load_cols] = df_hourly[load_cols].fillna(0)
    df_hourly['total_load'] = df_hourly[load_cols].sum(axis=1)
    df_main = df_hourly[['datetime', 'total_load']]

    # 2. Load Traffic
    df_traffic = pd.read_csv(FILE_TRAFFIC, sep=';', decimal=',')
    df_traffic['datetime'] = pd.to_datetime(df_traffic['Date_from'], format='%d.%m.%Y %H:%M')
    traffic_cols = df_traffic.select_dtypes(include=[np.number]).columns
    df_traffic['total_traffic'] = df_traffic[traffic_cols].sum(axis=1)
    
    # Merge Traffic
    df_main = pd.merge(df_main, df_traffic[['datetime', 'total_traffic']], on='datetime', how='left').fillna(0)

    # 3. Load Weather
    df_weather = pd.read_csv(FILE_WEATHER)
    df_weather['date'] = pd.to_datetime(df_weather['datetime']).dt.date
    
    # Merge Weather (Daily to Hourly)
    df_main['date'] = df_main['datetime'].dt.date
    df_main = pd.merge(df_main, df_weather[['date', 'temp', 'solar_rad']], on='date', how='left').fillna(0)
    
    # 4. Feature Engineering (Cyclical Time)
    df_main['hour_sin'] = np.sin(2 * np.pi * df_main['datetime'].dt.hour / 24)
    df_main['hour_cos'] = np.cos(2 * np.pi * df_main['datetime'].dt.hour / 24)
    
    # Sort and Index
    df_main = df_main.sort_values('datetime').reset_index(drop=True)
    
    # Features to use
    feature_cols = ['total_load', 'total_traffic', 'temp', 'solar_rad', 'hour_sin', 'hour_cos']
    return df_main, feature_cols

df, features = prepare_dataset()
print(f'Dataset Shape: {df.shape}')
print(f'Features: {features}')

## 4. Preprocessing for LSTM
Neural Networks require scaled data. We create sequences: using data from $t-24$ to $t$ to predict $t+1$.

In [ ]:
# 1. Split Train/Test (Time-based)
train_size = int(len(df) * 0.8)
train_df = df.iloc[:train_size]
test_df = df.iloc[train_size:]

# 2. Scale
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_df[features])
test_scaled = scaler.transform(test_df[features])

# 3. Create Sequences
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:i+seq_length]
        y = data[i+seq_length][0] # Target is 'total_load' (index 0)
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

X_train, y_train = create_sequences(train_scaled, SEQ_LENGTH)
X_test, y_test = create_sequences(test_scaled, SEQ_LENGTH)

# 4. Convert to PyTorch Tensors
train_loader = DataLoader(TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train)), 
                          shuffle=True, batch_size=BATCH_SIZE)
test_loader = DataLoader(TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test)), 
                         shuffle=False, batch_size=BATCH_SIZE)

print(f'Train Sequences: {X_train.shape}')

## 5. Defining the LSTM Model
We use a stacked LSTM architecture.

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size=1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Initialize hidden/cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

model = LSTMModel(input_size=len(features), hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

## 6. Training Loop
We train the model for a set number of epochs.

In [ ]:
losses = []
for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch).squeeze()
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(train_loader)
    losses.append(avg_loss)
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{EPOCHS}], Loss: {avg_loss:.4f}')

plt.plot(losses)
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.show()

## 7. Evaluation
We predict on the test set and inverse-transform the scaling to get real kW values.

In [ ]:
model.eval()
predictions = []
actuals = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        preds = model(X_batch).squeeze()
        predictions.extend(preds.numpy())
        actuals.extend(y_batch.numpy())

# Inverse Transform (Tricky part: Scaler expects all features)
# We create a dummy matrix to invert just the Load column
def inverse_transform_target(scaled_data, scaler, feature_index=0):
    dummy = np.zeros((len(scaled_data), len(features)))
    dummy[:, feature_index] = scaled_data
    return scaler.inverse_transform(dummy)[:, feature_index]

y_pred_real = inverse_transform_target(predictions, scaler)
y_test_real = inverse_transform_target(actuals, scaler)

# Metrics
mae = mean_absolute_error(y_test_real, y_pred_real)
rmse = np.sqrt(mean_squared_error(y_test_real, y_pred_real))

print(f'\nTest MAE: {mae:.2f} kW')
print(f'Test RMSE: {rmse:.2f} kW')

# Plotting (First 200 hours of test)
plt.figure(figsize=(12, 6))
plt.plot(y_test_real[:200], label='Actual Load')
plt.plot(y_pred_real[:200], label='Predicted Load (LSTM)')
plt.legend()
plt.title('Hourly Load Forecast (Test Set Sample)')
plt.show()